# Data Collection via Web Scraping
## Scrape Multiple Pages

In [5]:
from kaggle_secrets import UserSecretsClient
import requests
from requests.exceptions import HTTPError, RequestException
from lxml import html, etree
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import sys
import pandas as pd

### Step 0: Set up the MySQL database credentials
##### Clever Cloud database credentials 

In [6]:
user_secrets = UserSecretsClient()
username = user_secrets.get_secret("username")
password = user_secrets.get_secret("password")
host = user_secrets.get_secret("host")
port = user_secrets.get_secret("port")
database = user_secrets.get_secret("database")

### Step 1: Set up the connection string and 
### create the SQLAlchemy engine that connects to the MySQL database

In [7]:
connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string)

### Step 2: Define the base URL and initialize the page number 

In [8]:
base_url = 'https://books.toscrape.com/catalogue/page-{}.html'
page_number = 1
books = []

### Step 3: Fetch the current page

In [9]:
def fetch_page(url):
    """
    Send an HTTP GET request to the specified URL so as to fetch the current page. 
    Raise an exception for bad responses (e.g., HTTPError, URLError, ConnectionError, etc.) 
    and then handle them if they exist.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
        sys.exit(1)
    except RequestException as req_err:
        print(f'Request error occurred: {req_err}')
        sys.exit(1)

### Step 4: Set the correct encoding for the correct parsing of the characters

In [10]:
def set_encoding(response):
    """
    Get the Content-Type header, check if it contains the charset parameter.
    If so, then set the encoding accordingly; otherwise, default the encoding to UTF-8.
    """
    charset = 'charset='
    content_type = response.headers.get('Content-Type')

    if charset in content_type:
        encoding = content_type.split(charset)[-1]
    else:
        encoding = 'utf-8'

    response.encoding = encoding

    return response.text

### Step 5: Validate the HTML content

In [11]:
def validate_html(html_content):
    """
    Validate the HTML content. If the content is not well-formed, then raise an exception.
    """
    try: 
        document = html.fromstring(html_content)
        return document
    except (etree.XMLSyntaxError, ValueError) as e:
        print(f'HTML content is not well-formed: {e}')
        sys.exit(1)

### Step 6: Clean and parse the HTML content with BeautifulSoup

In [12]:
def clean_html(document):
    """
    Convert the lxml document back into the HTML string.
    Parse the converted HTML content using BeautifulSoup.
    Remove unnecessary tags (e.g., script, style).
    """
    html_string = etree.tostring(document, encoding='unicode', pretty_print=True)

    soup = BeautifulSoup(html_string, 'lxml')

    for script in soup(['script', 'style']):
        script.decompose()

    return soup

### Step 7: Get all of the books in the current page 

In [13]:
def get_books(soup):
    """
    Get all of the books in the current page.
    Get the book titles, prices and in stock features; store them in a list of dictionaries.
    """
    books_list = soup.find_all('article', class_='product_pod')

    if not books_list:
        print('No more books left!')
    else:
        for book in books_list:
            h3_tag = book.find('h3')
            title = h3_tag.find('a').get('title').strip() if h3_tag else 'No Title'
            price = book.find('p', class_='price_color').text.strip()
            in_stock = book.find('p', class_='instock availability').text.strip()

            book_dict = {'Title': title, 'Price': price, 'In Stock': in_stock}   
            books.append(book_dict)

### Step 8: Loop through the website pages to fetch them

In [14]:
while True:
    # Construt the URL for the current page
    url = base_url.format(page_number)

    # Send an HTTP GET request to the constructed URL
    response = fetch_page(url)

    # Set the correct encoding for proper parsing of the characters
    html_content = set_encoding(response)

    # Validate the HTML content
    document = validate_html(html_content)

    # Convert the lxml document back into HTML string
    # Clean and parse the HTML content
    soup = clean_html(document)

    # Get all of the books in the current page
    get_books(soup)

    # Increment the page number
    page_number += 1

    # Break the code after looping through all the pages
    if page_number > 50:
        break
    

In [15]:
len(books)

1000

### Step 9: Convert the list of dictionaries (containing books info) 
### into a Pandas DataFrame for easier manipulation

In [16]:
books_data = pd.DataFrame(books) 

In [17]:
books_data.head()

,Title,Price,In Stock
0,A Light in the Attic,£51.77,In stock
1,Tipping the Velvet,£53.74,In stock
2,Soumission,£50.10,In stock
3,Sharp Objects,£47.82,In stock
4,Sapiens: A Brief History of Humankind,£54.23,In stock


In [18]:
books_data.isnull().sum()

Title       0
Price       0
In Stock    0
dtype: int64

In [19]:
books_data.tail()

,Title,Price,In Stock
995,Alice in Wonderland (Alice's Adventures in Won...,£55.53,In stock
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",£57.06,In stock
997,A Spy's Devotion (The Regency Spies of London #1),£16.97,In stock
998,1st to Die (Women's Murder Club #1),£53.98,In stock
999,"1,000 Places to See Before You Die",£26.08,In stock


In [20]:
books_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     1000 non-null   object
 1   Price     1000 non-null   object
 2   In Stock  1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


### Step 10: Save the scraped data to a CSV file and a MySQL database

In [21]:
books_data.to_csv('books_1000', index=False)

In [22]:
books_data.to_sql('books', con=engine, if_exists='replace')

1000

### Step 11: Close the MySQL Database Connection

In [23]:
engine.dispose()